<a href="https://colab.research.google.com/github/FKz11/NLP/blob/main/lesson_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson-9

### Библиотеки:

In [1]:
import tensorflow as tf
import numpy as np

### Данные:

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path_to_file = '/content/gdrive/MyDrive/Colab Notebooks/evgenyi_onegin.txt'

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [5]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [9]:
examples_per_epoch

2841

In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


Сместим предложения для организации тренировочных данных

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [13]:
idx2char[input_example.numpy()]

array(['А', 'л', 'е', 'к', 'с', 'а', 'н', 'д', 'р', ' ', 'С', 'е', 'р',
       'г', 'е', 'е', 'в', 'и', 'ч', ' ', 'П', 'у', 'ш', 'к', 'и', 'н',
       '\n', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'Е', 'в', 'г', 'е', 'н',
       'и', 'й', ' ', 'О', 'н', 'е', 'г', 'и', 'н', '\n', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], dtype='<U1')

Перемешивать нельзя, так как потом мы будем использовать stateful в rnn

In [14]:
# Batch size
BATCH_SIZE = 64

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 1024

# Number of RNN units
rnn_units = embedding_dim

Возьмём модель с 4 lstm слоями со stateful=True, для переноса состояния через batchю А так же пробросим shortcut connections.

In [16]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units, batch_size):
        super().__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None])
                                 
        self.lstm1 = tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')
        self.lstm2 = tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')
        self.lstm3 = tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')
        self.lstm4 = tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        x1 = self.emb(x)
        x = self.lstm1(x1)
        x = self.lstm2(x)
        x2 = x + x1
        x = self.lstm3(x2)
        x = self.lstm4(x)
        x = x + x2
        return self.fc(x)

In [17]:
model = RNNgenerator(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "rn_ngenerator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  134144    
                                                                 
 lstm (LSTM)                 multiple                  8392704   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 lstm_2 (LSTM)               multiple                  8392704   
                                                                 
 lstm_3 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  134275    
                                                                 
Total params: 33,839,235
Trainable params: 33,839,235

In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '

Next Char Predictions: 
 "w,ЛDыКy)NЗqOМэyАияdmiЬk4PCiПЕшхь3тй.9MзПжfPpЬЗТTэA,Ц4йTwКпwuMсДhшq'Т5БжсP!А2zзКqHБg;УDУcкioыpL5b!ыП\n"


loss

In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.862646


In [23]:
model.compile(optimizer='adam', loss=loss)

Модель для предсказаний с размерностью пакета = 1

In [24]:
model_1 = RNNgenerator(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=1)

In [25]:
model_1.build(tf.TensorShape([1, None]))

In [26]:
for i in range(len(model.layers)):
  model_1.layers[i].set_weights(model.layers[i].get_weights())

Predict

In [27]:
def generate_text(model, start_string, temperature=1, num_generate=500):

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

CustomCallback

In [29]:
from keras.callbacks import Callback

class CustomCallback(Callback):

    def __init__(self, start_string, temperature, num_generate):
        super().__init__()
        self.start_string = start_string
        self.temperature = temperature
        self.num_generate = num_generate

    def on_epoch_end(self, epoch, logs=None):
        if epoch%5==0:
          for i in range(len(model_1.layers)):
            model_1.layers[i].set_weights(self.model.layers[i].get_weights())
          text = generate_text(model_1, self.start_string, self.temperature, self.num_generate)
          print('\n'+text)

fit

In [30]:
EPOCHS = 101
start_string='И вот идет уже '
temperature=1
num_generate=500
my_callback = CustomCallback(start_string, temperature, num_generate)

In [66]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=my_callback)

Epoch 1/101
 6/44 [===>..........................] - ETA: 14s - loss: 5.2604

44/44 [==============================] - ETA: 0s - loss: 2.4007
И вот идет уже                       Н      БД  н                   а о 
 идпояар                           СО гуясдесситоиокмлмл е,                                 Кь.ьйним друтнр  мур  и м                        аленбг.ет нхол  гитотхнзп                                        С           XПгодс
аулулс,гвlв.да ем                                     Кьор
 с  иокЛуто обц                     л   тлжамйи  олут нвт р иероти
нто  .вай 
дтгоявДн тбтвзи
уИен.:                                     Кл.оиаеоаюдсрйяуй нло
44/44 [==============================] - 29s 566ms/step - loss: 2.4007
Epoch 2/101
44/44 [==============================] - 17s 390ms/step - loss: 1.6328
Epoch 3/101
44/44 [==============================] - 18s 400ms/step - loss: 1.4373
Epoch 4/101
44/44 [==============================] - 17s 386ms/step - loss: 1.3358
Epoch 5/101
44/44 [==============================] - 17s 385ms/step - loss: 1.2898
Epoch 6/101
44/44

In [67]:
model.save('/content/gdrive/MyDrive/Colab Notebooks/model_generate_text')

*** checkpoint ***

In [36]:
from tensorflow import keras
model = keras.models.load_model('/content/gdrive/MyDrive/Colab Notebooks/model_generate_text', compile=False)

In [37]:
model.compile(optimizer='adam', loss=loss)

In [38]:
for i in range(len(model_1.layers)):
            model_1.layers[i].set_weights(model.layers[i].get_weights())
print(generate_text(model_1, start_string, temperature=1, num_generate=500))

И вот идет уже                    То видит он врагов забвенных,
                        Клеветников, и трусов злых,
        Постылой жизни мишура,
                        Мои успехи в вихре света,
                              Ученый малый, но педант:
                        Имел он счастливый талант
                        Простонародной старины,
                        И снов задумчивой души.
                        Без грамматической ошибки
                        Я русский Нашел, для небо моды,
           


Как зависит predict от T

In [93]:
temperature=2
print(generate_text(model_1, start_string, temperature, num_generate=500))

И вот идет уже                       Что ды, дады уздавался он.

                                     XXII

              И увлекательный и вредный?
                        Я не могу понять. Но вот
                    (Шушу модных на(суждений,
          i         Куть Везде после 5ноги мечтаний,
                        гостей не слушает она
                        На троть, может, на мог                    Monsieur lще, больше! гости,
                        Е суждено совсем иное...
                   ны своем


При увелечении T фразы становятся более случайными, но менее логичными

In [96]:
temperature=0.5
print(generate_text(model_1, start_string, temperature, num_generate=500))

И вот идет уже                       И вот уже трещат морозы
                        И серебрятся средь полей...
                                                         XXXII

                        Привычка усладила горе,
                           Поклонник мирных аонид,
                        О ты, чья память сохранит
                         И продолжительный присест.

                                      IX

                       и про девиц;
                        А нынче все мне темно, Таня:
     


Здесь фразы выглядят более логичными, но они очевино менее случайные и встречались в исходном тексте.

In [99]:
temperature=3
print(generate_text(model_1, start_string, temperature, num_generate=500))

И вот идет уже ЛАно
   BeНе.

         Еpi 
 rma этой сем"Iестословом,
   мVIrefI"
   X    V

     Или;
            7,
        стынный Онегине с кНовы,
   Но ь)
     LV. II

      C    
 учPреги Таньoваz На чеуты?.
    :
          Уженья, грусть, за мноМы
  ш, др"ХаВной
  И 4Перешел:
   чнд)
 м     тPе  хилосткого все.
 Свенча: карась,
       Журал;          страм?
    FPеt.
         Таить,
          Дынно этом мNанилужда,
   и6 стрH звонь тащет. При сутрюою 8одна;
    здалем
     лужой -
   Предскав1и W.
- Я 


При слишком высокой температуре уже проподает логика

In [109]:
temperature=0.00001
print(generate_text(model_1, start_string, temperature, num_generate=500))

И вот идет уже                       И вот уже трещат морозы
                        И серебрятся средь полей...
                                                 XXIII

                        Что было следствием свиданья?
                            Поешь бог ведает кого,
                        И свод элегий драгоценный
                        У                          XXVIII

                        С душою, полной сожалений,
           Что намарал я свой портрет,
                        Как Байрон, гордос


In [110]:
temperature=0.00001
print(generate_text(model_1, start_string, temperature, num_generate=500))

И вот идет уже                       И вот уже трещат морозы
                        И серебрятся средь полей...
                                                 XXIII

                        Что было следствием свиданья?
                            Поешь бог ведает кого,
                        И свод элегий драгоценный
                        У                          XXVIII

                        С душою, полной сожалений,
           Что намарал я свой портрет,
                        Как Байрон, гордос


При слишком низкой температуре, вариация генерации пропадает и модель начинает строить одну и ту же последовательность.

In [86]:
import re

In [113]:
re.search('И вот уже трещат морозы\n                        И серебрятся средь полей...', text)

<re.Match object; span=(133141, 133216), match='И вот уже трещат морозы\n                        >

In [114]:
text[133141:133216]

'И вот уже трещат морозы\n                        И серебрятся средь полей...'

Тоесть при маленькой температуре модель старается брать уже заученные фразы, а не генерировать новые. Но при этом все равно генерация фразы длиной 500 не будет ровно такой же, которая была в исходном тексте.